---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [105]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind


# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [106]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

def get_list_of_university_towns1():
import re
    #open the file
    f = open('university_towns.txt', 'r')
    lines = f.readlines()
    lines = [i.rstrip() for i in lines]
    f.close()
    #make the list
    name_list = []
    for line in lines:
        if '[edit]' in line:
            state_name = line[:-6]
        elif '(' in line:
            region_name = line[:line.find('(') - 1]
            name_list.append((state_name, region_name))
        else:#不要忘了没有（）的地区们
            region_name = line
            name_list.append((state_name, region_name))
    df = pd.DataFrame(name_list, columns = ['State', 'RegionName'])
    return df
get_list_of_university_towns()
    
get_list_of_university_towns1()

In [162]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    import re
    #open the file
    f = open('university_towns.txt', 'r')
    lines = f.readlines()
    lines = [i.rstrip() for i in lines]#最重要的一步 在读取文件时一定要按行来进行去空格！！！！！！！！
    f.close()
    #make the list
    name_list = []
    for line in lines:
        if '[edit]' in line:
            state_name = line[:-6]
        elif '(' in line:
            region_name = line[:line.find('(') - 1]
            name_list.append((state_name, region_name))
        else:#不要忘了没有（）的地区们
            region_name = line
            name_list.append((state_name, region_name))
    df = pd.DataFrame(name_list, columns = ['State', 'RegionName'])
    return df
get_list_of_university_towns()



,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [163]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel('gdplev.xls', skiprows = 220, usecols = 'E,G', header = None)#原题中让使用chained的GDP因此（E，G）
    GDP.columns = ['Quarter', 'GDP']
    for j in range(1, len(GDP['GDP']-2)):
        if GDP.iloc[j,1] < GDP.iloc[(j-1), 1] and GDP.iloc[(j-1),1] < GDP.iloc[(j-2), 1]: 
            startdate = GDP.iloc[j-3,0] #-3是因为这个iloc[]是从0开始的 因此原本应该后退两个 但是在这里要多减1
    return startdate
get_recession_start()

'2008Q3'

In [164]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel('gdplev.xls', skiprows = 220, usecols = 'E,G', header = None)
    GDP.columns = ['Quarter', 'GDP']
    start = get_recession_start()
    GDP = GDP[GDP['Quarter'] > start]
    for j in range(1, len(GDP['GDP'])):
        if GDP.iloc[j,1] > GDP.iloc[(j-1), 1] and GDP.iloc[(j-1),1] > GDP.iloc[(j-2), 1]: 
            enddate = GDP.iloc[j,0]  
            break #只取第一次的 而不是最后一次的 我们只需要第一次符合有条件的内容
    return enddate
get_recession_end()

'2009Q4'

In [165]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel('gdplev.xls', skiprows = 220, usecols = 'E,G', header = None)
    GDP.columns = ['Quarter', 'GDP']
    start = get_recession_start()
    end = get_recession_end()
    GDP = GDP[GDP['Quarter'] < end]
    GDP = GDP[GDP['Quarter'] > start]
    GDP = GDP.set_index('Quarter')
    return GDP['GDP'].argmin()
get_recession_bottom()

/Users/baiyiwei/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  # This is added back by InteractiveShellApp.init_path()


'2009Q2'

In [166]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing = pd.read_csv('City_Zhvi_AllHomes.csv', header = 0)
    housing = housing.drop(columns =['SizeRank','RegionType', 'Metro', 'CountyName','StateName', 'RegionID'], axis =1)
    housing = housing.replace({'State':states}).sort_values(['State','RegionName'])
    housing = housing.set_index(['State','RegionName'])
    lst = []
    for t in housing.columns.tolist():
        if 1999 < int(t[:4]) < 2017:
           lst.append(t) 
    lst = lst[:-3]
    housing = housing[lst]
    housing = housing.groupby(pd.PeriodIndex(housing.columns, freq = 'Q'), axis = 1).mean()
    housing.columns = housing.columns.values.astype(str)
    return housing
convert_housing_data_to_quarters().loc[[('Ohio','Akron'),('Ohio','Dayton')]].loc[:,['2010Q3','2015Q2','2016Q4']]


2010Q3        2015Q2  2016Q4
State RegionName                                    
Ohio  Akron       69953.333333  55045.666667     NaN
      Dayton      43191.666667  40703.333333     NaN

In [190]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    start = get_recession_start()
    end = get_recession_end()
    housing = convert_housing_data_to_quarters()
    uni_town = get_list_of_university_towns()
    bottom = get_recession_bottom()
    
    housing['Ratio'] = housing[start] / housing[bottom]
    #get all info of 
    uni_town['uni_town'] = True
    all_house = pd.merge(housing, uni_town, how = 'outer', left_index = True, right_on = ['State', 'RegionName'])
    all_house['uni_town'] = all_house['uni_town'].replace({np.NaN: False})#因为NaN并不是string而是numpy根据null得到的内容因此要用np.NaN来指代
    uni_house = all_house[all_house['uni_town'] == True]
    non_house = all_house[all_house['uni_town'] == False]
    
    st, p = ttest_ind(uni_house['Ratio'], non_house['Ratio'], nan_policy = 'omit')
    if p < 0.01:
        different = True
    else:
        different = False
    if uni_house['Ratio'].dropna().mean() > non_house['Ratio'].dropna().mean():
        better = 'non-university town'
    else:
        better = 'university tow'
    return (different, p, better)
run_ttest()

/Users/baiyiwei/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  # This is added back by InteractiveShellApp.init_path()


(False, 0.7399478338009744, 'non-university town')